In [88]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import WebDriverException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import json
import time
import random
import datetime

# 公開資訊觀測站

## 上市 / 上櫃 / 興櫃 累計與當月營業收入統計表

In [16]:
def getMonthlyRevenue(year, month):
    companyMarketCategories = ['sii', 'otc', 'rotc']

    columns=['公司代號', 
             '公司名稱', 
             '當月營收', 
             '上月營收', 
             '去年當月營收', 
             '上月比較增減(%)', 
             '去年同月增減(%)', 
             '當月累計營收', 
             '去年累計營收', 
             '前期比較增減(%)', 
             '備註']

    monthlyRevenueDataFrame = pd.DataFrame(columns=columns)

    for companyMarketCategory in companyMarketCategories:
        url = 'https://mops.twse.com.tw/nas/t21/' + companyMarketCategory + '/t21sc03_' + str(year) + '_' + str(month) + '_0.html'
        try:
            response = requests.get(url)
            response.encoding = 'big5'

            htmlTextParsed = BeautifulSoup(response.text, 'html.parser')
            allCompaniesByIndustrialCategoryTables = htmlTextParsed.find_all(name='table', attrs={'border': 5})

            # the last table is summation
            for allCompaniesPerIndustrialCategoryTable in allCompaniesByIndustrialCategoryTables[:-1]:
                allCompaniesPerIndustrialCategoryRows = allCompaniesPerIndustrialCategoryTable.find_all(name='tr', attrs={'align':'right'})
                # the last row is summation
                for singleCompanyRow in allCompaniesPerIndustrialCategoryRows[:-1]:
                    singleCompanyColumns = singleCompanyRow.find_all('td')
                    singleCompanyDictionary = {}
                    for i in range(len(singleCompanyColumns)):
                        singleCompanyDictionary[columns[i]] = singleCompanyColumns[i].text.strip()
                    monthlyRevenueDataFrame.loc[len(monthlyRevenueDataFrame)] = singleCompanyDictionary
            
        except requests.RequestException as errorMessage:
            print(errorMessage)
        
        # pause for website access limitation
        time.sleep(random.randint(1, 5))

    monthlyRevenueDataFrame.to_csv('./Data/monthly_revenue_' + str(year) + '_' + str(month) + '.csv', index=False)
    return monthlyRevenueDataFrame.shape

In [17]:
monthlyRevenueDataFrameShape = getMonthlyRevenue(113,7)
print(monthlyRevenueDataFrameShape)

(1867, 11)


## 上市 / 上櫃 / 興櫃 財務分析資料查詢彙總表

In [20]:
def getAnnualFinantialAnalysis(year):
    url = 'https://mops.twse.com.tw/mops/web/ajax_t51sb02'

    companyMarketCategories = ['sii', 'otc', 'rotc']

    columns = ['公司代號','公司簡稱','負債佔資產比率(%)','長期資金佔不動產、廠房及設備比率(%)','流動比率(%)','速動比率(%)','利息保障倍數(%)','應收款項週轉率(次)','平均收現日數','存貨週轉率(次)','平均銷貨日數','不動產、廠房及設備週轉率(次)','總資產週轉率(次)','資產報酬率(%)','權益報酬率(%)','稅前純益佔實收資本比率(%)','純益率(%)','每股盈餘(元)','現金流量比率(%)','現金流量允當比率(%)','現金再投資比率(%)']
    annualFinantialAnalysisDataFrame = pd.DataFrame(columns=columns)

    dataForPostRequest = {
        'encodeURIComponent': 1,
        'step': 1,
        'TYPEK': 'sii',
        'year': year,
        'firstin': 1,
        'off': 1,
        'ifrs': 'Y',
    }

    for companyMarketCategory in companyMarketCategories:
        dataForPostRequest['TYPEK'] = companyMarketCategory
        try:
            response = requests.post(url, data=dataForPostRequest)
            
            htmlTextParsed = BeautifulSoup(response.text, 'html.parser')
            allCompaniesTable = htmlTextParsed.find(name='table', attrs={'class': 'hasBorder'})

            allCompaniesRows = allCompaniesTable.find_all('tr', attrs={'class': ['even', 'odd']})

            for singleCompanyRow in allCompaniesRows:
                singleCompanyColumns = singleCompanyRow.find_all('td')
                singleCompanyDictionary = {}
                for i in range(len(singleCompanyColumns)):
                    singleCompanyDictionary[columns[i]] = singleCompanyColumns[i].text.strip()
                annualFinantialAnalysisDataFrame.loc[len(annualFinantialAnalysisDataFrame)] = singleCompanyDictionary
            
        except requests.RequestException as errorMessage:
            print(errorMessage)
        
        # pause for website access limitation
        time.sleep(random.randint(1, 5))    
    annualFinantialAnalysisDataFrame.to_csv('./Data/annual_financial_analysis_' + str(year) + '.csv', index=False)
    return annualFinantialAnalysisDataFrame.shape

In [21]:
annualFinantialAnalysisDataFrameShape = getAnnualFinantialAnalysis(112)
print(annualFinantialAnalysisDataFrameShape)

(2107, 21)


## 上市 / 上櫃 / 興櫃 綜合損益表 / 資產負債表 / 現金流量表

In [17]:
def getSheet(year, season):
    url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb'
    url_suffices = ['04', '05', '20']
    sheet_names = ['income_statement', 'balanced_sheet', 'cashflow_statement']

    companyMarketCategories = ['sii', 'otc', 'rotc']

    dataForPostRequest = {
        'encodeURIComponent': 1,
        'step': 1,
        'firstin': 1,
        'off': 1,
        'isQuery': 'Y',
        'TYPEK': 'sii',
        'year': str(year),
        'season': '0'+str(season),
    }

    for url_suffix, sheet_name in zip(url_suffices, sheet_names):
        SheetDataFrame = pd.DataFrame()
        for companyMarketCategory in companyMarketCategories:
            dataForPostRequest['TYPEK'] = companyMarketCategory

            try:
                response = requests.post(url+url_suffix, data=dataForPostRequest)

                htmlTextParsed = BeautifulSoup(response.text, 'html.parser')
                allCompaniesByIndustrialCategoryTables = htmlTextParsed.find_all(name='table', attrs={'class':'hasBorder'})


                for allCompaniesPerIndustrialCategoryTable in allCompaniesByIndustrialCategoryTables:
                    columnNames = allCompaniesPerIndustrialCategoryTable.find_all('th')
                    columns = []
                    for columnName in columnNames:
                        columns.append(columnName.text.strip())

                    SheetPerIndustrialCategoryDataFrame = pd.DataFrame(columns=columns)

                    allCompaniesPerIndustrialCategoryRows = allCompaniesPerIndustrialCategoryTable.find_all(name='tr', attrs={'class':['even', 'odd']})
                    for singleCompanyRow in allCompaniesPerIndustrialCategoryRows:
                        singleCompanyColumns = singleCompanyRow.find_all('td')
                        singleCompanyDictionary = {}
                        for i in range(len(singleCompanyColumns)):
                            singleCompanyDictionary[columns[i]] = singleCompanyColumns[i].text.strip()
                        SheetPerIndustrialCategoryDataFrame.loc[len(SheetPerIndustrialCategoryDataFrame)] = singleCompanyDictionary
                        
                    SheetDataFrame = pd.concat([SheetDataFrame, SheetPerIndustrialCategoryDataFrame], ignore_index=True)

                    

            except requests.RequestException as errorMessage:
                print(errorMessage)

            # pause for website access limitation
            time.sleep(random.randint(1, 5))
    
        SheetDataFrame.to_csv('./Data/quarterly_' + sheet_name + '_' + str(year) + '_' + str(season) + '.csv', index=False)
        print(SheetDataFrame.shape)

In [ ]:
getSheet(112, 1)

# 台灣證券交易所

In [14]:
def getClosingPrice_sii(year, month, day):
    date = datetime.datetime(year, month, day).strftime('%Y%m%d')

    url = 'https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX?date=' + date + '&type=ALLBUT0999&response=json&_=' + str(round(time.time() * 1000))

    try:
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})

        response_json = json.loads(response.text)
        df = pd.DataFrame()

        for table in response_json['tables']:
            if table:
                df_ = pd.DataFrame(table['data'], columns=table['fields'])
                df = pd.concat([df, df_], ignore_index=True)

        df.to_csv('./Data/daily_closing_price_sii_' + str(year-1911) + '_' + str(month) + '_' + str(day) + '.csv')
        print(df.shape)
    except requests.RequestException as errorMessage:
                    print(errorMessage)

In [15]:
getClosingPrice_sii(2024, 9, 12)

113年09月12日 價格指數(臺灣證券交易所)
價格指數(跨市場)
價格指數(臺灣指數公司)
報酬指數(臺灣證券交易所)
報酬指數(跨市場)
報酬指數(臺灣指數公司)
113年09月12日 大盤統計資訊
漲跌證券數合計
113年09月12日 每日收盤行情(全部(不含權證、牛熊證))
(1544, 28)


In [65]:
datetime.datetime.now().day

13

# 證券櫃檯買賣中心

In [92]:
def getClosingPrice_sii():
    url = 'https://www.tpex.org.tw/web/emergingstock/lateststats/new.htm?l=zh-tw'

    try:
        driver = webdriver.Chrome()
        driver.get(url)
        _ = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'odd')))

        htmlTextParsed = BeautifulSoup(driver.page_source, 'html.parser')

        header = htmlTextParsed.find('thead')
        columnNames = header.find_all('th')
        columns = []
        for columnName in columnNames:
            columns.append(columnName.text.strip())

        df = pd.DataFrame(columns=columns)

        rows = htmlTextParsed.find_all(name='tr', attrs={'class':['even', 'odd']})
        for row in rows:
            singleCompanyColumns = row.find_all('td')
            singleCompanyList = []
            for i in range(len(singleCompanyColumns)):
                singleCompanyList.append(singleCompanyColumns[i].text.strip())
            df.loc[len(df)] = singleCompanyList

        currentdate = datetime.datetime.now()

        df.to_csv('./Data/daily_closing_price_rotc_' + str(currentdate.year-1911) + '_' + str(currentdate.month) + '_' + str(currentdate.day) + '.csv')
        print(df.shape)

        driver.quit()
    except TimeoutException as errorMessage:
        print(errorMessage)
    except WebDriverException as errorMessage:
        print(errorMessage)

In [93]:
getClosingPrice_sii()

(329, 16)
